In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
train_data = pd.read_excel('Data_Train.xlsx')
test_data = pd.read_excel('Data_Test.xlsx')
submission_data = pd.read_excel('Sample_Submission.xlsx')

In [3]:
train_data.Delivery_Time.value_counts()

30 minutes     7406
45 minutes     2665
65 minutes      923
120 minutes      62
20 minutes       20
80 minutes       14
10 minutes        4
Name: Delivery_Time, dtype: int64

In [4]:
set(test_data.Restaurant.unique())-set(train_data.Restaurant.unique())
#(11094, 9)

{'ID_8113',
 'ID_4184',
 'ID_2396',
 'ID_3704',
 'ID_4798',
 'ID_8290',
 'ID_3139',
 'ID_5056',
 'ID_3762',
 'ID_3936',
 'ID_4919',
 'ID_4547',
 'ID_854',
 'ID_4852',
 'ID_2082',
 'ID_2630',
 'ID_69',
 'ID_2827',
 'ID_2961',
 'ID_6834',
 'ID_7902',
 'ID_5977',
 'ID_1202',
 'ID_4661',
 'ID_6045',
 'ID_7327',
 'ID_3677',
 'ID_67',
 'ID_7477',
 'ID_1395',
 'ID_6303',
 'ID_235',
 'ID_4719',
 'ID_8388',
 'ID_6483',
 'ID_3972',
 'ID_2423',
 'ID_680',
 'ID_4288',
 'ID_2840',
 'ID_4374',
 'ID_1567',
 'ID_6961',
 'ID_7387',
 'ID_6963',
 'ID_3471',
 'ID_1880',
 'ID_51',
 'ID_458',
 'ID_8516',
 'ID_3016',
 'ID_7317',
 'ID_7565',
 'ID_331',
 'ID_6911',
 'ID_5826',
 'ID_123',
 'ID_5917',
 'ID_8501',
 'ID_396',
 'ID_879',
 'ID_4339',
 'ID_3172',
 'ID_3628',
 'ID_4178',
 'ID_4265',
 'ID_8377',
 'ID_851',
 'ID_7320',
 'ID_2381',
 'ID_446',
 'ID_5034',
 'ID_7685',
 'ID_1542',
 'ID_5606',
 'ID_3606',
 'ID_2574',
 'ID_5730',
 'ID_3944',
 'ID_3820',
 'ID_4613',
 'ID_4277',
 'ID_7983',
 'ID_2477',
 'ID_710

In [5]:
train_data[train_data.Delivery_Time.isin(['80 minutes'])].head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
558,ID_1368,"Yerawada, Pune, Maharashtra",Bakery,₹200,₹50,4.2,242,94,80 minutes
2546,ID_6134,Mumbai Central,"Chinese, North Indian, Thai, Fast Food, Seafood",₹400,₹50,3.8,149,74,80 minutes
3451,ID_2837,"D-Block, Sector 63, Noida","Mithai, Chinese, South Indian",₹150,₹50,3.5,191,33,80 minutes
4542,ID_2057,Mumbai Central,"Desserts, Bakery",₹100,₹50,4.0,58,44,80 minutes
5279,ID_4741,"Dockyard Road, Mumbai CST Area","North Indian, Seafood, Fast Food, Continental,...",₹400,₹99,3.5,550,263,80 minutes


In [6]:
# 30 minutes     2119
# 45 minutes      635
# 120 minutes      10
# 65 minutes        9
# 80 minutes        1

train_data = train_data[train_data.Delivery_Time.isin(['30 minutes','45 minutes','120 minutes','65 minutes','80 minutes','20 minutes'])]

In [7]:
train_data.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,65 minutes
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",₹250,₹99,3.7,176,95,30 minutes
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",₹200,₹99,3.2,521,235,65 minutes


In [8]:
test_data.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews
0,ID_2842,"Mico Layout, Stage 2, BTM Layout,Bangalore","North Indian, Chinese, Assamese",₹350,₹50,4.2,361,225
1,ID_730,"Mico Layout, Stage 2, BTM Layout,Bangalore","Biryani, Kebab",₹100,₹50,NEW,-,-
2,ID_4620,"Sector 1, Noida",Fast Food,₹100,₹50,3.6,36,16
3,ID_5470,"Babarpur, New Delhi, Delhi","Mithai, North Indian, Chinese, Fast Food, Sout...",₹200,₹50,3.6,66,33
4,ID_3249,"Sector 1, Noida","Chinese, Fast Food",₹150,₹50,2.9,38,14


In [9]:
len(train_data.Location.unique())

35

In [10]:
len(test_data.Location.unique())

35

### Work around Cuisines features

In [11]:
restaurant_typ_list=train_data.Cuisines.apply(lambda x : x.split(',')).tolist()+test_data.Cuisines.apply(lambda x : x.split(',')).tolist()
restaurant_name_type = list(set([j.strip() for sub in restaurant_typ_list for j in sub]))
for t in restaurant_name_type:
    train_data[t]= train_data.Cuisines.apply(lambda x : 0 if t not in x else 1)
    test_data[t]= test_data.Cuisines.apply(lambda x : 0 if t not in x else 1)

# train_data['train_flag'] = 1
# test_data['train_flag'] = 0
# test_data['Delivery_Time'] = -1
# merge_train_test = pd.concat([train_data,test_data])

### Work around rating Features

In [12]:
rating_list=train_data.Rating.tolist()+test_data.Rating.tolist()
rating_name = list(set(rating_list))
for t in rating_name:
    train_data[t]= train_data.Rating.apply(lambda x : 0 if t not in x else 1)
    test_data[t]= test_data.Rating.apply(lambda x : 0 if t not in x else 1)

#train_data['train_flag'] = 1
#test_data['train_flag'] = 0
#test_data['Delivery_Time'] = -1
#merge_train_test = pd.concat([train_data,test_data])

In [13]:
rating_name

['2.2',
 '3.5',
 '2.5',
 '2.3',
 '3.9',
 '3.1',
 'NEW',
 '2.8',
 'Temporarily Closed',
 '3.3',
 '4.4',
 '4.1',
 '4.2',
 '3.2',
 'Opening Soon',
 '4.6',
 '2.7',
 '3.6',
 '4.8',
 '4.0',
 '-',
 '4.7',
 '4.5',
 '2.6',
 '4.9',
 '2.1',
 '3.7',
 '2.4',
 '3.8',
 '2.9',
 '3.4',
 '4.3',
 '3.0']

In [14]:
submission_data.head()

,Delivery_Time
0,120 minutes
1,20 minutes
2,20 minutes
3,65 minutes
4,45 minutes


In [15]:

# #Remove row which have 'for' in Average_Cost
# train_data = train_data[train_data.Average_Cost!='for']
# #Conver into int
# #train_data['Average_Cost'] = train_data['Average_Cost'].apply(lambda x : int(x.replace(',','').replace('₹','')))

# #test_data['Average_Cost'] = test_data['Average_Cost'].apply(lambda x : int(x.replace(',','').replace('₹','')))

# train_data['Minimum_Order'] = train_data['Minimum_Order'].apply(lambda x : int(x.replace(',','').replace('₹','')))

# test_data['Minimum_Order'] = test_data['Minimum_Order'].apply(lambda x : int(x.replace(',','').replace('₹','')))



In [16]:
#train_data['new_rating'] = train_data['Rating']
#train_data.groupby(['Delivery_Time','Rating']).agg({'new_rating':'count'}).reset_index().sort_values(by=['new_rating'],ascending=False)

In [17]:
train_data.Rating.unique()

array(['3.5', '3.6', '3.7', '3.2', '3.8', '4.0', '3.9', '4.2', '-', '2.8',
       '3.0', '3.3', '3.1', '4.7', '3.4', '4.1', 'NEW', '2.9', '4.6',
       '4.3', '2.6', '4.5', '4.4', '4.8', '2.4', '2.7', '2.5',
       'Opening Soon', '2.2', '4.9', '2.3', '2.1', 'Temporarily Closed'],
      dtype=object)

In [18]:
def data_fill(df):
    #Remove row which have 'for' in Average_Cost
    df = df[df.Average_Cost!='for']
    #Conver into int
    
    df['Average_Cost'] = df['Average_Cost'].apply(lambda x : int(x.replace(',','').replace('₹','')))
    df['Minimum_Order'] = df['Minimum_Order'].apply(lambda x : int(x.replace(',','').replace('₹','')))
    
    df['log_avg_cost'] = np.log(df['Average_Cost'])
    df['log_min_order'] = np.log(df['Minimum_Order'])
    
    df['Rating_Bins'] = df['Rating'].apply(lambda x : x if x in ['Opening Soon','NEW','-','Temporarily Closed']
                                                      else 'Rating_<=1.5' if float(x)<=1.5
                                                      else 'Rating_1.5-2.0' if float(x)>1.5 and float(x)<=2.0 
                                                      else 'Rating_2.0-2.5' if float(x)>2.0 and float(x)<=2.5
                                                      else 'Rating_2.5-3.0' if float(x)>2.5 and float(x)<=3.0
                                                      else 'Rating_3.0-3.5' if float(x)>3.0 and float(x)<=3.5
                                                      else 'Rating_3.5-4.0' if float(x)>3.5 and float(x)<=4.0
                                                      else 'Rating_>4.0' if float(x)>4.0 else x)
    df['new_rating'] = df['Rating'].apply(lambda x : -1 if x in ['Opening Soon','NEW','-','Temporarily Closed']
                                                      else float(x))
    
    
    df.loc[df.Votes=='-','Votes'] = -1
    df.loc[df.Reviews=='-','Reviews']=-1
    
    df['Votes'] = df.Votes.astype('int')
    df['Reviews'] = df.Reviews.astype('int')
    
    df['TOT_Rating_By_Votes']=df['Rating'].apply(lambda x : -1 if x in ['Opening Soon','NEW','-','Temporarily Closed'] else float(x))
    df['TOT_Rating_By_Votes'] = np.where(df['TOT_Rating_By_Votes']==-1,-1,df['TOT_Rating_By_Votes']*df['Votes'])
    
    df['TOT_Rating_By_Review']=df['Rating'].apply(lambda x : -1 if x in ['Opening Soon','NEW','-','Temporarily Closed'] else float(x))
    df['TOT_Rating_By_Review'] = np.where(df['TOT_Rating_By_Review']==-1,-1,df['TOT_Rating_By_Review']*df['Reviews'])
    
    df['TOT_Rating_By_Vote_Review']=df['Rating'].apply(lambda x : -1 if x in ['Opening Soon','NEW','-','Temporarily Closed'] else float(x))
    df['TOT_Rating_By_Vote_Review'] = np.where(df['TOT_Rating_By_Vote_Review']==-1,-1,df['TOT_Rating_By_Vote_Review']*(df['Reviews']+df['Votes']))

    df['NUMBER_OF_Cuisines'] = df['Cuisines'].apply(lambda x : len(x.split(',')))
    
    df['AVG_TOT_COST'] = df['Average_Cost']*df['Votes']
    df['AVG_TOT_COST'] = df['AVG_TOT_COST'].apply(lambda x : -1 if x<0 else x)
    df['AVG_TOT_MIN_ORDER'] = df['Minimum_Order']*df['Votes']
    df['AVG_TOT_MIN_ORDER'] = df['AVG_TOT_MIN_ORDER'].apply(lambda x : -1 if x<0 else x)
    
    df = pd.get_dummies(df,columns=['Rating_Bins'])
    df = pd.get_dummies(df,columns=['Location'])
    return df

In [19]:
train_data_pre = data_fill(train_data)
test_data_pre = data_fill(test_data)

In [20]:
train_data_pre.Delivery_Time.value_counts()

# 30 minutes     2119
# 45 minutes      635
# 120 minutes      10
# 65 minutes        9
# 80 minutes        1

30 minutes     7405
45 minutes     2665
65 minutes      923
120 minutes      62
20 minutes       20
80 minutes       14
Name: Delivery_Time, dtype: int64

In [21]:
#train_data_pre = train_data_pre[~(train_data_pre.Delivery_Time.isin(['10 minutes','80 minutes','20 minutes','120 minutes']))]

In [22]:
train_data_pre.Delivery_Time.value_counts()
# 30 minutes     2119
# 45 minutes      635
# 120 minutes      10
# 65 minutes        9
# 80 minutes        1

30 minutes     7405
45 minutes     2665
65 minutes      923
120 minutes      62
20 minutes       20
80 minutes       14
Name: Delivery_Time, dtype: int64

In [23]:
drop_col = ['Restaurant','Cuisines','Average_Cost','Minimum_Order','Rating','Delivery_Time','Rating_Bins_Temporarily Closed']
target_col =['Delivery_Time']
X,y = train_data_pre.drop(columns=drop_col,axis=1),train_data_pre['Delivery_Time']

from sklearn.utils import class_weight
class_weights = list(class_weight.compute_class_weight('balanced',
                                             np.unique(y),
                                             y))

y_dic={}
for w,c_y in zip(class_weights,np.unique(y)):
    y_dic[c_y]=w
w_array = np.ones(y.shape[0], dtype = 'float')
for i, val in enumerate(y):
    w_array[i] = y_dic[val]

In [24]:
# 30 minutes     7405
# 45 minutes     2665
# 65 minutes      923
# 120 minutes      62
# 80 minutes       14
label_dic = {'30 minutes':1,'45 minutes':2,'120 minutes':3,'65 minutes':4,'80 minutes':0,'20 minutes':5}
label_dic_rev = {1:'30 minutes',2:'45 minutes',3:'120 minutes',4:'65 minutes',0:'80 minutes',5:'20 minutes'}
new_y = y.apply(lambda x :label_dic[x] )

In [ ]:
import lightgbm as lgb

print("Light Gradient Boosting Classifier: ")
lgbm_params =  {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 7,
    'metric': ['multi_error'],
    "learning_rate": 0.05,
     "num_leaves": 60,
     "max_depth": 9,
     "feature_fraction": 0.45,
     "bagging_fraction": 0.3,
     "reg_alpha": 0.15,
     "reg_lambda": 0.15,
#      "min_split_gain": 0,
      "min_child_weight": 0
                }

lgb_cv = lgb.cv(
    params = lgbm_params,
    train_set = lgtrain,
    num_boost_round=2000,
    stratified=True,
    nfold = 5,
    verbose_eval=50,
    seed = 23,
    early_stopping_rounds=75)

loss = lgbm_params["metric"][0]
optimal_rounds = np.argmin(lgb_cv[str(loss) + '-mean'])
best_cv_score = min(lgb_cv[str(loss) + '-mean'])

print("\nOptimal Round: {}\nOptimal Score: {} + {}".format(
    optimal_rounds,best_cv_score,lgb_cv[str(loss) + '-stdv'][optimal_rounds]))

results = results.append({"Rounds": optimal_rounds,
                          "Score": best_cv_score,
                          "STDV": lgb_cv[str(loss) + '-stdv'][optimal_rounds],
                          "LB": None,
                          "Parameters": lgbm_params}, ignore_index=True)
if Home is True:
    with open('results.csv', 'a') as f:
        results.to_csv(f, header=False)